### Installing Dependencies

**surprise란?** 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지.
사이킷런과 유사한 API 와 프레임워크를 제공. 

**surprise 주요 모듈**
surprise는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있음. 네번째 칼럼부터는 로딩을 수행하지 않는다. 



In [ ]:
!pip install surprise

### Importing Libraries

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix # csr_matrix: 희소 행렬 (대부분의 요소가 0임.)
# CountVectorizer: 단어들의 count(출현 빈도)로 여러 문서들을 벡터화
# TfidVectorizer: Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter("ignore")

In [ ]:
anime_info_df = pd.read_csv('../input/anime-recommendation-database-2020/anime.csv')
anime_desc_df = pd.read_csv("../input/anime-recommendation-database-2020/anime_with_synopsis.csv")
rating_df = pd.read_csv('../input/anime-recommendations-database/rating.csv')

In [ ]:
anime_info_df.head(5)

In [ ]:
anime_df = pd.merge(anime_desc_df, anime_info_df[["MAL_ID","Type","Popularity","Members","Favorites"]], on = "MAL_ID")
anime_df.head(10)

In [ ]:
anime_df.info()

In [ ]:
anime_df["Score"].describe()

In [ ]:
anime_df = anime_df[(anime_df["Score"]!="Unknown")]
anime_df.shape

## Content Filtering

In [ ]:
anime_df['sypnopsis'] = anime_df['sypnopsis'].fillna('')

TF-IDF란?
* TF(term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값. 이 값이 높을수록 문서에서 중요함. (다른 문서에 자주 등장하면 단어의 중요도는 낮아짐.)
* DF(document frequency)는 문서 빈도로, 이 값의 역수를 IDF라고 한다. 
* TF-IDF 는 TF와 IDF를 곱한 값으로, 점수가 높은 단어일수록 다른 문서에는 많지 않고 해당 문서에서 자주 등장하는 단어임을 의미한다.

* ngram_range: The lower and upper boundary of the range of n-values for different n-grams to be extracted.
* stop_words: 무시할 수 있는 단어. 보통 영어의 관사나 접속사, 한국어의 조사 등이 해당 됨.

In [ ]:
tfidf = TfidfVectorizer(analyzer="word", ngram_range=(1,2), min_df=0, stop_words = "english")
tfidf_matrix = tfidf.fit_transform(anime_df['sypnopsis'])
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

In [ ]:
anime_df = anime_df.reset_index()
titles = anime_df["Name"]
indices = pd.Series(anime_df.index, index = anime_df["Name"])

In [ ]:
def content_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:31]
    anime_indices = [i[0] for i in sim_scores]

    anime_lst = anime_df.iloc[anime_indices][["Name","Members","Score"]]
    favorite_count = anime_lst[anime_lst["Members"].notnull()]["Members"].astype("int")
    score_avg = anime_lst[anime_lst["Score"].notnull()]["Score"].astype("float")
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_lst[(anime_lst["Members"]>=m)&(anime_lst["Members"].notnull())&(anime_lst["Score"].notnull())]
    qualified["Members"] = qualified["Members"].astype("int")
    qualified["Score"] = qualified["Score"].astype("float")
    def weighted_rating(x):
        v = x["Members"]
        R = x["Score"]
        return (v/(v+m) * R) + (m/(m+v) * C)

    qualified["wr"] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values("wr", ascending=False).head(10)

    return qualified

In [ ]:
content_recommendations("Naruto").head(10)

## collaborative Filtering

In [ ]:
rating_df.head(10)

In [ ]:
rating_df['rating'].value_counts()

In [ ]:
rating_df = rating_df[(rating_df["rating"] != -1)]
rating_df.head(5)

In [ ]:
reader = Reader()
rating_data = Dataset.load_from_df(rating_df, reader)
svd = SVD() # 특이값 분해 https://losskatsu.github.io/linear-algebra/svd/#11-%EC%A0%95%EB%A6%AC

In [ ]:
trainset = rating_data.build_full_trainset() # 전체 데이터를 학습 데이터로 사용

In [ ]:
svd.fit(trainset)

In [ ]:
svd.predict(1, 356, 5)

## Hybrid Filtering

In [ ]:
id_map = anime_df[["MAL_ID"]]
id_map["id"] = list(range(1, anime_df.shape[0]+1,1))
id_map = id_map.merge(anime_df[["MAL_ID", "Name"]], on = "MAL_ID").set_index("Name")
id_map

In [ ]:
indices_map = id_map.set_index("id")
indices_map

In [ ]:
def hybrid_recommendations(user_id,title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]    
    anime_indices = [i[0] for i in sim_scores]
            
    anime_lst = anime_df.iloc[anime_indices][['MAL_ID','Name', 'Members', 'Score','Genres']]
    favorite_count = anime_lst[anime_lst['Members'].notnull()]['Members'].astype('int')
    score_avg = anime_lst[anime_lst['Score'].notnull()]['Score'].astype('float')
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_lst[(anime_lst['Members'] >= m) & (anime_lst['Members'].notnull()) & (anime_lst['Score'].notnull())]    
    qualified['Members'] = qualified['Members'].astype('int')
    qualified['Score'] = qualified['Score'].astype('float')
    def weighted_rating(x):
        v = x['Members']
        R = x['Score']
        return (v/(v+m) * R) + (m/(m+v) * C)   
    
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(30)    
    
    # hybrid에서 추가된 부분. 
    qualified[['id']] = list(range(1,qualified.shape[0]+1,1))  
    qualified['est'] = qualified['id'].apply(lambda x: svd.predict(user_id, indices_map.loc[x]['MAL_ID']).est)
    qualified = qualified.sort_values('est', ascending=False)
    result = qualified[['MAL_ID','Name','Genres','Score']]
    return result.head(10)    

In [ ]:
hybrid_recommendations(8, 'Trigun')